In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from common import get_train, get_test, feat_eng, add_street_sections, get_parking
%matplotlib inline

In [16]:
train_df = get_train()
test_df = get_test()

train_df = feat_eng(train_df)
test_df = feat_eng(test_df)

Time it took: 0.14214801788330078
Time it took: 0.08444690704345703
adding additional features to test and/or train ...

loading parking data 1.7M
loading address data 30K
calculating closest joins
merging
complete
fixing neighborhood string formatting
merging in neighborhood information
Time it took: 0.7101459503173828
adding additional features to test and/or train ...

loading parking data 1.7M
loading address data 30K
calculating closest joins
merging
complete
fixing neighborhood string formatting
merging in neighborhood information
Time it took: 0.6179640293121338


In [50]:
parking_def = get_parking()

loading cached copy
Time it took: 2.1573381423950195


In [51]:
parking_df, train_df, test_df = add_street_sections(train_df, test_df)

loading cached copy
Time it took: 2.1488640308380127
Identifying key streets...
Filtering down the parking data ...
measuring parking coords within FROM TO bounds ...
Adding street section ids to train and parking ...
Complete


### Mean target encoding with parking

In [52]:
#def parking_mean_encoding(train_df, parking_df)
grp_cols = ['STR_SEC', 'morn_night','dow','day']
max_spots = parking_df.groupby(grp_cols).size().reset_index()
max_spots.columns = grp_cols + ['spot_ct']
max_spots = max_spots.groupby('STR_SEC')[['spot_ct']].max().reset_index()
max_spots.head(50)
max_spots.columns = ['STR_SEC', 'MAX_SPOTS']

grp_cols = ['STR_SEC', 'morn_night','dow','day']
pct_tbl = parking_df.groupby(grp_cols).size().reset_index()
pct_tbl.columns = grp_cols + ['spot_ct']
pct_tbl = pct_tbl.merge(max_spots, how='left', on='STR_SEC')
pct_tbl['avail_pct'] = 1 - pct_tbl['spot_ct']/pct_tbl['MAX_SPOTS']

grp_cols = ['STR_SEC','morn_night','dow']
avg_pct_tbl = pct_tbl.groupby(grp_cols)['avail_pct'].agg([np.mean, np.max]).reset_index()

In [53]:
joined = train_df.merge(avg_pct_tbl, how = 'left', on =grp_cols)


In [54]:
joined.head()

,Street,From,To,Date,Time,Real.Spots,Street.Length,any_spot,holiday,day,...,med_age,pop,twn_avg_value,twn_pct,STREET,STR_SEC_x,STR_SEC_y,STR_SEC,mean,amax
0,Mission Street,25th Street,26th Street,2014-01-07,16:19,4,179.132970,1,0,7,...,38.4,25825.0,1048521.0,18.5,MISSION STREET,0,0,0,0.915323,0.931452
1,Polk Street,Ellis Street,Olive Street,2014-01-18,20:42,0,52.740210,0,0,18,...,42.5,14670.0,718750.0,2.0,POLK STREET,30,30,30,NaN,NaN
2,Van Ness Avenue,Geary Boulevard,Myrtle Street,2014-01-18,20:39,0,52.517840,0,0,18,...,47.1,14331.0,1041250.0,6.6,VAN NESS AVENUE,38,38,38,0.990291,0.990291
3,Van Ness Avenue,Bush Street,Fern Street,2014-01-18,20:38,0,52.405315,0,0,18,...,45.9,1395.0,NaN,NaN,VAN NESS AVENUE,52,52,52,NaN,NaN
4,Van Ness Avenue,Daniel Burnham Court,Post Street,2014-01-18,20:38,0,52.191193,0,0,18,...,41.5,9194.0,NaN,NaN,VAN NESS AVENUE,46,46,46,NaN,NaN


In [58]:
joined['mean'].isna()

0       False
1        True
2       False
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11      False
12       True
13       True
14       True
15       True
16       True
17       True
18       True
19      False
20      False
21       True
22      False
23       True
24       True
25      False
26       True
27      False
28       True
29       True
        ...  
1107     True
1108     True
1109     True
1110     True
1111     True
1112    False
1113    False
1114     True
1115     True
1116     True
1117     True
1118     True
1119     True
1120     True
1121     True
1122     True
1123     True
1124     True
1125     True
1126    False
1127     True
1128     True
1129     True
1130    False
1131     True
1132    False
1133     True
1134     True
1135     True
1136     True
Name: mean, Length: 1137, dtype: bool